Google Stocks

In [98]:
import pandas as pd
import numpy as np
from neuralprophet import NeuralProphet, set_log_level
import math

import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import ipywidgets as widgets
from ipywidgets import interact_manual

plotting_backend = "plotly-static"

In [3]:
df = pd.read_csv(
    "https://raw.githubusercontent.com/ourownstory/neuralprophet-data/main/datasets/multivariate/ER_Europe_subset_10nodes.csv"
)
df["ID"] = df["ID"].astype(str)
df["ds"] = pd.to_datetime(df["ds"])
# # use one year for faster training
df = df[df["ID"] =="1"]
df=df.drop(columns=["solar","solar_fcs","ID"])
df.head()

,ds,y
0,2012-01-01 00:00:00,75.6549
1,2012-01-01 01:00:00,70.9958
2,2012-01-01 02:00:00,66.6382
3,2012-01-01 03:00:00,62.9920
4,2012-01-01 04:00:00,61.0699


In [4]:
df.shape

(26304, 2)

Distrubution plots

In [6]:
fig = px.line(df, x=df["ds"], y=df["y"])
fig.show()